In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from keras import models
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
import os
import copy

In [2]:
def cross_validate_and_select_best_params(model, X, y, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    scores = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': [],
        'roc_auc': []
    }

    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # print(f"y_train shape = {y_train.shape}, y_test shape = {y_test.shape}")
        history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

        y_pred_prob = model.predict(X_test)
        y_pred = np.argmax(y_pred_prob, axis=1)
        y_test = np.argmax(y_test, axis=1) 
        
        scores['accuracy'].append(accuracy_score(y_test, y_pred))
        scores['precision'].append(precision_score(y_test, y_pred, average='macro'))
        scores['recall'].append(recall_score(y_test, y_pred, average='macro'))
        scores['f1'].append(f1_score(y_test, y_pred, average='macro'))
        scores['roc_auc'].append(roc_auc_score(y_test, y_pred_prob, multi_class='ovr', average='macro'))  # ROC AUC 多分类问题

    avg_scores = {key: np.mean(value) for key, value in scores.items()}
    return avg_scores

In [2]:
import os
import pickle
import numpy as np

def load_cifar10_data(data_dir):
    X = []
    Y = []

    # Load training data batches
    for i in range(1, 6):
        with open(os.path.join(data_dir, f'data_batch_{i}'), 'rb') as file:
            batch = pickle.load(file, encoding='latin1')
            X.append(batch['data'])
            Y.extend(batch['labels'])

    # Combine training data
    X = np.concatenate(X)

    # Load test data
    with open(os.path.join(data_dir, 'test_batch'), 'rb') as file:
        test_batch = pickle.load(file, encoding='latin1')
        X_test = test_batch['data']
        Y_test = test_batch['labels']

    # Combine test data
    X = np.vstack((X, X_test))
    Y.extend(Y_test)

    # Convert labels to numpy array
    Y = np.array(Y)

    # Note: Removing the reshaping operation
    # X = X.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # (N, H, W, C)

    # Load label names from batches.meta
    with open(os.path.join(data_dir, 'batches.meta'), 'rb') as file:
        meta = pickle.load(file, encoding='latin1')
        label_names = meta['label_names']

    return X, Y, label_names


In [3]:
data_dir = 'cifar-10-batches-py'  # path directory
X_combined, y_combined, meta = load_cifar10_data(data_dir)

In [5]:
def build_simple_cnn(input_shape, num_classes, learning_rate=0.0005, num_filter=16, filter_size=(3,3), dropout_rate=0.5):
    model = models.Sequential()
    model.add(layers.Conv2D(num_filter, filter_size, activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_classes, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [6]:
print(y_combined.shape)

(60000,)


In [64]:
input_shape = (32, 32, 3)
num_classes = 10

num_filters=[8]
learning_rates=[0.001, 0.0005, 0.0015, 0.01, 0.005]
filter_sizes=[(1,1), (3,3), (5,5), (7,7), (9,9)]
dropout_rates=[0, 0.1, 0.3, 0.5, 0.75]

highest_config=[]
highest_accuracy=0
for learning_rate in learning_rates:
    highest_num_filter = 0
    highest_num_filter_accuracy=0
    best_num_filter = False
    for num_filter in num_filters:
        highest_filter_size = (0,0)
        highest_filter_size_accuracy=0
        best_filter_size = False
        for filter_size in filter_sizes:
            highest_dropout_rate = -1
            highest_dropout_rate_accuracy=0
            for dropout_rate in dropout_rates:
                current_config = [learning_rate, num_filter, filter_size, dropout_rate]
                model = build_simple_cnn(input_shape, num_classes, learning_rate, num_filter, filter_size, dropout_rate)

                X_combined_reshaped = X_combined.reshape(-1, 32, 32, 3)
                y_combined_categorical = to_categorical(y_combined, num_classes=10)
                X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0

                avg_scores = cross_validate_and_select_best_params(model, X_combined_reshaped, y_combined_categorical)
                print("current_config: ", current_config)
                print(avg_scores)

                if avg_scores['accuracy'] > highest_dropout_rate_accuracy:
                    highest_dropout_rate_accuracy = avg_scores['accuracy']
                    highest_dropout_rate = current_config[3]
                else:
                    print("Last dropout_rate is best, break")
                    break

                if avg_scores['accuracy'] >= highest_filter_size_accuracy:
                    highest_filter_size_accuracy = avg_scores['accuracy']
                    highest_filter_size = current_config[2]
                    best_filter_size = False
                elif highest_filter_size_accuracy > avg_scores['accuracy'] and filter_size > highest_filter_size:
                    print("Last filter_size is best, break")
                    best_filter_size = True

                if avg_scores['accuracy'] >= highest_num_filter_accuracy:
                    highest_num_filter_accuracy = avg_scores['accuracy']
                    highest_num_filter = current_config[1]
                    best_num_filter = False
                elif highest_num_filter_accuracy > avg_scores['accuracy'] and num_filter > highest_num_filter:
                    print("Last num_filter is best, break")
                    best_num_filter = True


                if avg_scores['accuracy'] > highest_accuracy:
                    highest_config = current_config
                    highest_accuracy = avg_scores['accuracy']
                    print("highest_config: ", highest_config)
                    print("highest_accuracy: ", highest_accuracy)

            if best_filter_size:
                break
        if best_num_filter:
            break
            
print("highest_config: ", highest_config)
print("highest_accuracy: ", highest_accuracy)



Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.3947 - loss: 1.7181 - val_accuracy: 0.4337 - val_loss: 1.6551
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5795 - loss: 1.2057 - val_accuracy: 0.5570 - val_loss: 1.2822
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6373 - loss: 1.0387 - val_accuracy: 0.5552 - val_loss: 1.3060
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6891 - loss: 0.9051 - val_accuracy: 0.5617 - val_loss: 1.3006
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7365 - loss: 0.7768 - val_accuracy: 0.5665 - val_loss: 1.3299
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7160 - loss: 0.8515 - val_accuracy: 0.7347 - val_loss: 0.7591
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7709 - loss: 0.6997 - val_accuracy: 0.7132 - val_loss: 0.8119
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8081 

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3748 - loss: 1.7763 - val_accuracy: 0.3875 - val_loss: 1.7234
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5659 - loss: 1.2290 - val_accuracy: 0.5590 - val_loss: 1.2756
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6191 - loss: 1.0828 - val_accuracy: 0.5773 - val_loss: 1.2268
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6556 - loss: 0.9835 - val_accuracy: 0.5764 - val_loss: 1.2384
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6917 - loss: 0.8901 - val_accuracy: 0.5792 - val_loss: 1.2352
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.6760 - loss: 0.9541 - val_accuracy: 0.7282 - val_loss: 0.7943
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7107 - loss: 0.8445 - val_accuracy: 0.6952 - val_loss: 0.8626
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.727

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4043 - loss: 1.7186 - val_accuracy: 0.4821 - val_loss: 1.5053
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5849 - loss: 1.1801 - val_accuracy: 0.5613 - val_loss: 1.2387
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6484 - loss: 1.0042 - val_accuracy: 0.5591 - val_loss: 1.2608
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6925 - loss: 0.8826 - val_accuracy: 0.5932 - val_loss: 1.2184
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7389 - loss: 0.7617 - val_accuracy: 0.5991 - val_loss: 1.2242
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7235 - loss: 0.8097 - val_accuracy: 0.7347 - val_loss: 0.7485
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7696 - loss: 0.6750 - val_accuracy: 0.7078 - val_loss: 0.8478
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.808

...

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 68s 180ms/step - accuracy: 0.4019 - loss: 2.6407 - val_accuracy: 0.4993 - val_loss: 1.6091
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 183ms/step - accuracy: 0.6103 - loss: 1.1038 - val_accuracy: 0.5432 - val_loss: 1.3210
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 72s 191ms/step - accuracy: 0.6849 - loss: 0.9109 - val_accuracy: 0.5842 - val_loss: 1.2197
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 186ms/step - accuracy: 0.7447 - loss: 0.7357 - val_accuracy: 0.5655 - val_loss: 1.3891
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/step - accuracy: 0.8054 - loss: 0.5693 - val_accuracy: 0.5612 - val_loss: 1.6522
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/step - accuracy: 0.7880 - loss: 0.6513 - val_accuracy: 0.7977 - val_loss: 0.5710
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 186ms/step - accuracy: 0.8511 - loss: 0.4475 - val_accuracy: 0.8051 - val_loss: 0.5623
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 71s 188ms/step - accuracy: 0.3655 - loss: 2.5755 - val_accuracy: 0.4544 - val_loss: 1.7289
Epoch 2/5


KeyboardInterrupt: 

In [7]:
input_shape = (32, 32, 3)
num_classes = 10

num_filters=[8, 16, 32, 64, 128]
learning_rates=[0.0005]
filter_sizes=[(3,3)]
dropout_rates=[0]

highest_config=[]
highest_accuracy=0
for learning_rate in learning_rates:
    highest_num_filter = 0
    highest_num_filter_accuracy=0
    best_num_filter = False
    for num_filter in num_filters:
        highest_filter_size = (0,0)
        highest_filter_size_accuracy=0
        best_filter_size = False
        for filter_size in filter_sizes:
            highest_dropout_rate = -1
            highest_dropout_rate_accuracy=0
            for dropout_rate in dropout_rates:
                current_config = [learning_rate, num_filter, filter_size, dropout_rate]
                model = build_simple_cnn(input_shape, num_classes, learning_rate, num_filter, filter_size, dropout_rate)

                X_combined_reshaped = X_combined.reshape(-1, 32, 32, 3)
                y_combined_categorical = to_categorical(y_combined, num_classes=10)
                X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0

                avg_scores = cross_validate_and_select_best_params(model, X_combined_reshaped, y_combined_categorical)
                print("current_config: ", current_config)
                print(avg_scores)

                if avg_scores['accuracy'] > highest_dropout_rate_accuracy:
                    highest_dropout_rate_accuracy = avg_scores['accuracy']
                    highest_dropout_rate = current_config[3]
                else:
                    print("Last dropout_rate is best, break")
                    break

                if avg_scores['accuracy'] >= highest_filter_size_accuracy:
                    highest_filter_size_accuracy = avg_scores['accuracy']
                    highest_filter_size = current_config[2]
                    best_filter_size = False
                elif highest_filter_size_accuracy > avg_scores['accuracy'] and filter_size > highest_filter_size:
                    print("Last filter_size is best, break")
                    best_filter_size = True

                if avg_scores['accuracy'] >= highest_num_filter_accuracy:
                    highest_num_filter_accuracy = avg_scores['accuracy']
                    highest_num_filter = current_config[1]
                    best_num_filter = False
                elif highest_num_filter_accuracy > avg_scores['accuracy'] and num_filter > highest_num_filter:
                    print("Last num_filter is best, break")
                    best_num_filter = True


                if avg_scores['accuracy'] > highest_accuracy:
                    highest_config = current_config
                    highest_accuracy = avg_scores['accuracy']
                    print("highest_config: ", highest_config)
                    print("highest_accuracy: ", highest_accuracy)

            if best_filter_size:
                break
        if best_num_filter:
            break
            
print("highest_config: ", highest_config)
print("highest_accuracy: ", highest_accuracy)

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.3952 - loss: 1.7335 - val_accuracy: 0.5036 - val_loss: 1.4665
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5656 - loss: 1.2387 - val_accuracy: 0.5377 - val_loss: 1.3129
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6258 - loss: 1.0728 - val_accuracy: 0.5402 - val_loss: 1.3355
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6732 - loss: 0.9519 - val_accuracy: 0.5773 - val_loss: 1.2310
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7062 - loss: 0.8485 - val_accuracy: 0.5780 - val_loss: 1.2565
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6999 - loss: 0.8698 - val_accuracy: 0.7002 - val_loss: 0.8461
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7429 - loss: 0.7541 - val_accuracy: 0.6879 - val_loss: 0.8898
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.775

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.4137 - loss: 1.7158 - val_accuracy: 0.5240 - val_loss: 1.4677
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.5972 - loss: 1.1388 - val_accuracy: 0.5787 - val_loss: 1.2086
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6656 - loss: 0.9545 - val_accuracy: 0.5938 - val_loss: 1.1747
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7189 - loss: 0.8130 - val_accuracy: 0.5881 - val_loss: 1.2325
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7617 - loss: 0.6919 - val_accuracy: 0.5909 - val_loss: 1.2755
375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.7519 - loss: 0.7535 - val_accuracy: 0.7524 - val_loss: 0.7058
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.7966 - loss: 0.6114 - val_accuracy: 0.7653 - val_loss: 0.6730
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy:

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step - accuracy: 0.4216 - loss: 1.7028 - val_accuracy: 0.5110 - val_loss: 1.5184
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.6207 - loss: 1.0925 - val_accuracy: 0.5846 - val_loss: 1.1708
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.6921 - loss: 0.8930 - val_accuracy: 0.6084 - val_loss: 1.1332
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.7438 - loss: 0.7483 - val_accuracy: 0.6153 - val_loss: 1.1556
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.7892 - loss: 0.6273 - val_accuracy: 0.6180 - val_loss: 1.1878
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.7681 - loss: 0.7050 - val_accuracy: 0.7571 - val_loss: 0.6759
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - accuracy: 0.8210 - loss: 0.5406 - val_accuracy: 0.7735 - val_loss: 0.6484
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - a

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 33s 85ms/step - accuracy: 0.4245 - loss: 1.7192 - val_accuracy: 0.5148 - val_loss: 1.6256
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 31s 84ms/step - accuracy: 0.6229 - loss: 1.0893 - val_accuracy: 0.5916 - val_loss: 1.1556
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 32s 84ms/step - accuracy: 0.6929 - loss: 0.8836 - val_accuracy: 0.6133 - val_loss: 1.1531
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 32s 85ms/step - accuracy: 0.7427 - loss: 0.7526 - val_accuracy: 0.6174 - val_loss: 1.1588
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 32s 84ms/step - accuracy: 0.7942 - loss: 0.6063 - val_accuracy: 0.6109 - val_loss: 1.2377
375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 32s 84ms/step - accuracy: 0.7771 - loss: 0.6856 - val_accuracy: 0.7705 - val_loss: 0.6545
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 32s 86ms/step - accuracy: 0.8296 - loss: 0.5171 - val_accuracy: 0.7855 - val_loss: 0.6151
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 32s 84ms/step - a

c:\Users\Deyu0\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 72s 189ms/step - accuracy: 0.4048 - loss: 1.9629 - val_accuracy: 0.5115 - val_loss: 1.7083
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 183ms/step - accuracy: 0.6258 - loss: 1.0764 - val_accuracy: 0.5987 - val_loss: 1.1500
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 69s 185ms/step - accuracy: 0.7031 - loss: 0.8551 - val_accuracy: 0.6123 - val_loss: 1.1566
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 186ms/step - accuracy: 0.7701 - loss: 0.6818 - val_accuracy: 0.6031 - val_loss: 1.2262
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/step - accuracy: 0.8253 - loss: 0.5222 - val_accuracy: 0.6144 - val_loss: 1.2664
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/step - accuracy: 0.7997 - loss: 0.6242 - val_accuracy: 0.8204 - val_loss: 0.5164
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 70s 187ms/step - accuracy: 0.8651 - loss: 0.4230 - val_accuracy: 0.8140 - val_loss: 0.5278
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 68s 183ms/

In [4]:
X_combined_reshaped = X_combined.reshape(-1, 32, 32, 3)
y_combined_categorical = to_categorical(y_combined, num_classes=10)
X_combined_reshaped = X_combined_reshaped.astype('float32') / 255.0

In [7]:
# GFNet simple
def global_filter_layer(x, filters, kernel_size=3, strides=1):
    # simple Conv2d
    x = layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    return x

# build GFNet model
def build_gfnet(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = global_filter_layer(inputs, filters=64, kernel_size=7, strides=2)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = global_filter_layer(x, filters=128, kernel_size=3)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(256, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, x)
    return model

input_shape = (32, 32, 3)
num_classes = 10

model = build_gfnet(input_shape, num_classes)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()

history = model.fit(X_combined_reshaped, y_combined_categorical, 
                    epochs=10, 
                    batch_size=64, 
                    validation_split=0.1)

print(f"Training finished. Accuracy: {history.history['accuracy'][-1]}")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 16, 16, 64)     │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 16, 16, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_8 (ReLU)                  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 382,858 (1.46 MB)

 Trainable params: 381,962 (1.46 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 70s 81ms/step - accuracy: 0.3486 - loss: 1.7906 - val_accuracy: 0.3282 - val_loss: 2.0693
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 70s 82ms/step - accuracy: 0.4846 - loss: 1.4371 - val_accuracy: 0.2935 - val_loss: 2.5365
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 70s 82ms/step - accuracy: 0.5326 - loss: 1.3120 - val_accuracy: 0.3577 - val_loss: 1.7651
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 68s 81ms/step - accuracy: 0.5665 - loss: 1.2188 - val_accuracy: 0.3998 - val_loss: 1.9559
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 69s 82ms/step - accuracy: 0.5896 - loss: 1.1599 - val_accuracy: 0.4965 - val_loss: 1.4356
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 68s 81ms/step - accuracy: 0.6119 - loss: 1.0995 - val_accuracy: 0.5488 - val_loss: 1.2741
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 67s 80ms/step - accuracy: 0.6287 - loss: 1.0525 - val_accuracy: 0.5823 - val_loss: 1.1678
Epoch 8/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 68s 80ms/step - accuracy: 0.6451 - loss: 1.0053 - 

In [10]:
# 定义Swin Transformer模型
class SwinTransformerBlock(layers.Layer):
    def __init__(self, num_heads, window_size, mlp_ratio=4.0):
        super(SwinTransformerBlock, self).__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=64)
        self.mlp = models.Sequential([
            layers.Dense(int(64 * mlp_ratio), activation='gelu'),
            layers.Dense(64)
        ])
        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)

    def call(self, inputs):
        # Multi-head self-attention
        attn = self.attention(inputs, inputs)
        x = self.norm1(inputs + attn)  # Residual connection

        # MLP
        mlp_output = self.mlp(x)
        return self.norm2(x + mlp_output)  # Residual connection

class SwinTransformer(models.Model):
    def __init__(self, num_classes):
        super(SwinTransformer, self).__init__()
        self.conv = layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')  # 增加卷积层
        self.block1 = SwinTransformerBlock(num_heads=4, window_size=4)
        self.block2 = SwinTransformerBlock(num_heads=4, window_size=4)
        self.flatten = layers.Flatten()
        self.dense = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv(inputs)  # 通过卷积层进行通道数调整
        x = tf.reshape(x, (-1, x.shape[1], x.shape[2], 64))  # 可能需要根据卷积后输出形状调整
        x = self.block1(x)
        x = self.block2(x)
        x = self.flatten(x)
        return self.dense(x)

model = SwinTransformer(num_classes=10)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
history = model.fit(X_combined_reshaped, y_combined_categorical, batch_size=64, epochs=10, validation_split=0.2)

# 保存模型
model.save('swin_transformer_cifar10.h5')
print(f"Training finished. Accuracy: {history.history['accuracy'][-1]}")

Epoch 1/10
 14/750 ━━━━━━━━━━━━━━━━━━━━ 54:02 4s/step - accuracy: 0.1334 - loss: 29.3850

KeyboardInterrupt: 